<div align="center" style=" font-size: 80%; text-align: center; margin: 0 auto">
<img src="https://raw.githubusercontent.com/Explore-AI/Pictures/master/Python-Notebook-Banners/Exercise.png"  style="display: block; margin-left: auto; margin-right: auto;";/>
</div>

# Exercise: Decision trees
© ExploreAI Academy

In this exercise, we will test our knowledge of the fundamental concepts of decision trees by implementing a decision tree regression model and analysing its performance metrics.

By the end of this exercise, you should be able to:
* Implement a decision tree regression model.

# Initial data exploration

We will be using our agriculture dataset to explore the effect climate change has on crops in Maji Ndogo. As a refresher of our data, examine the data dictionary.

# Data dictionary

**1. Geographic features**

- **Field_ID:** A unique identifier for each field (BigInt).
 
- **Elevation:** The elevation of the field above sea level in metres (Float).

- **Latitude:** Geographical latitude of the field in degrees (Float). (DUMMY VARIABLE – the simulation might have created a relationship)

- **Longitude:** Geographical longitude of the field in degrees (Float). (DUMMY VARIABLE – the simulation might have created a relationship)

- **Location:** Province the field is in (Text).

- **Slope:** The slope of the land in the field (Float).

**2. Weather features**

- **Rainfall:** Amount of rainfall in the area in mm (Float).

- **Min_temperature_C:** Average minimum temperature recorded in Celsius (Float). (DUMMY VARIABLE)

- **Max_temperature_C:** Average maximum temperature recorded in Celsius (Float). (DUMMY VARIABLE)

- **Ave_temps:** Average temperature in Celsius (Float).

**3. Soil and crop features**

- **Soil_fertility:** A measure of soil fertility where 0 is infertile soil and 1 is very fertile soil (Float).

- **Soil_type:** Type of soil present in the field (Text).

- **pH:** pH level of the soil, which is a measure of how acidic/basic the soil is (Float).

**4. Farm management features**

- **Field_ID:** Corresponding field identifier (BigInt).

- **Pollution_level:** Level of pollution in the area where 0 is unpolluted and 1 is very polluted (Float).

- **Plot_size:** Size of the plot in the field (Ha) (Float). (DUMMY VARIABLE)

- **Crop_type:** Type of crop chosen for cultivation (Text).

- **Annual_yield:** Annual yield from the field (Float). This is the total output of the field. The field size and type of crop will affect the annual yield. (DUMMY VARIABLE – removed)

<br>

**5. Target variable**
- **Standard_yield:** Standardised yield expected from the field, normalised per crop (Float). This is independent of field size or crop type. Multiplying this number by the field size and average crop yield will give the Annual_yield.

---

Let's import our database again, like we did last time. We won't use the weather data so it is commented out.

**Important:** Ensure that `data_ingestion.py` file and the `field_data_processor.py` files are stored in the same folder as your notebook, and check that the database file is created correctly, otherwise the data import will fail.
 
[Download files here](https://github.com/Explore-AI/Public-Data/raw/master/Maji_Ndogo/modules.zip)


In [2]:
import math

p_slow = 0.5
p_fast = 0.5

P_slow = 0.5 * math.log(0.5, 2)
P_fast = 0.5 * math.log(0.5, 2)
entropy = P_slow + P_fast
print(entropy)

-1.0


In [ ]:
# Read the database, and clean the data using the processing modules we built.

import re
import numpy as np
import pandas as pd
from field_data_processor import FieldDataProcessor
# from weather_data_processor import WeatherDataProcessor
import logging 

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')

config_params = {
    "sql_query": """
            SELECT *
            FROM geographic_features
            LEFT JOIN weather_features USING (Field_ID)
            LEFT JOIN soil_and_crop_features USING (Field_ID)
            LEFT JOIN farm_management_features USING (Field_ID)
            """,
    "db_path": 'sqlite:///Maji_Ndogo_farm_survey_small.db',
    "columns_to_rename": {'Annual_yield': 'Crop_type', 'Crop_type': 'Annual_yield'},
    "values_to_rename": {'cassaval': 'cassava', 'wheatn': 'wheat', 'teaa': 'tea'},
    "weather_csv_path": "https://raw.githubusercontent.com/Explore-AI/Public-Data/master/Maji_Ndogo/Weather_station_data.csv",
    "weather_mapping_csv": "https://raw.githubusercontent.com/Explore-AI/Public-Data/master/Maji_Ndogo/Weather_data_field_mapping.csv",
    "regex_patterns" : {
            'Rainfall': r'(\d+(\.\d+)?)\s?mm',
            'Temperature': r'(\d+(\.\d+)?)\s?C',
            'Pollution_level': r'=\s*(-?\d+(\.\d+)?)|Pollution at \s*(-?\d+(\.\d+)?)'
            },
}
# Ignoring the field data for now.
field_processor = FieldDataProcessor(config_params)
field_processor.process()
field_df = field_processor.df

# We're not going to use the weather data this time, so we'll ignore it.
# weather_processor = WeatherDataProcessor(config_params)
# weather_processor.process()
# weather_df = weather_processor.weather_df

dataset = field_df.drop("Weather_station", axis=1)

Let's take a quick look at our dataset to ensure it imported correctly.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, r2_score
import pandas as pd
import numpy as np
from sklearn.feature_selection import SelectKBest, f_regression
import seaborn as sns
import matplotlib.pyplot as plt

Before diving into our analysis, it's crucial to ensure the integrity of our dataset and that the data are still as we expect it to be.

In [ ]:
# Validate the data
# !pip install pytest

dataset.to_csv('sampled_field_df.csv', index=False)



import os# Define the file paths
field_csv_path = 'sampled_field_df.csv'

# Delete sampled_field_df.csv if it exists
if os.path.exists(field_csv_path):
    os.remove(field_csv_path)
    print(f"Deleted {field_csv_path}")
else:
    print(f"{field_csv_path} does not exist.")

## Exercises

### Exercise 1 

To begin our exploration of the dataset, we will use a decision tree to explore how all of the variables affect our crop yields.

Given our dataset containing information on environmental factors and crop yields, how can we train a decision tree model to predict crop yields accurately?

Fit a decision tree regressor model to the entire dataset.

Use all predictor variables (excluding non-numeric columns) to predict `Standard_yield`. 

Set the `max_depth` to 4 and the `random_state` to 42 for reproducibility.

Considering the **MSE** and **R²** values, how effective is the decision tree in predicting crop yields using all of our variables?

In [ ]:
#Your code here

### Exercise 2

While the model shows some predictive capability with a moderate R² value, there is still room for improvement, particularly in reducing the MSE and potentially increasing the explanatory power of the model. This can be achieved through further feature selection, hyperparameter tuning, or exploring alternative modelling techniques.

Let's start with feature selection. What environmental factors have the highest correlation with crop yields, and how can we use this information to improve the predictive performance of our model?

Conduct feature selection using a **correlation matrix** to select relevant features for predicting `Standard_yield`.

Visualise the matrix to better identify the features with the highest correlation with `Standard_yield`.

Regarding features to retain and eliminate, what do the results suggest? Have you thought about the independent features that might be correlated? 


In [ ]:
#Your code here

### Exercise 3
Now that we know which features could work best for our model, let's run our model again on a reduced dataset.

Reduce the dataset to only include `Pollution_level`, `Min_temperature_C`, `Longitude`, and `pH`.

Refit the decision tree regressor model using the reduced dataset and evaluate the model's performance on the test set to determine if there's an improvement in MSE and R² values.

Does reducing the dataset lead to an improvement in the predictive performance of our model?

Run the model on different combinations of the reduced dataset. Is there a combination of variables or a single variable that improves the MSE and R²?

In [ ]:
#Your code here

### Exercise 4
Now that we have attempted feature engineering, let's move on to parameter tuning. What is the optimal depth for the decision tree model that maximises predictive performance while avoiding overfitting?

Use a loop to fit decision tree models with depths ranging from 1 to 10 and assess the MSE and R² scores for each depth.
Analyse the results to identify the depth that maximises R² while avoiding significant increases in MSE, indicating optimal model complexity.

In [ ]:
#Your code here

## Solutions

### Exercise 1

In [ ]:

# Drop non-numeric columns
non_numeric_columns = ['Location', 'Soil_type', 'Crop_type']
#split data into dependent and independent variables
X = dataset.drop(columns=['Standard_yield'] + non_numeric_columns)
y = dataset['Standard_yield']
    
# Split the dataset into training and testing data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    
# Train the model
model = DecisionTreeRegressor(max_depth=4, random_state=42).fit(X_train, y_train)
predictions = model.predict(X_test)
    
# Evaluate the model
mse = mean_squared_error(y_test, predictions)
r2 = r2_score(y_test, predictions)
    
result = {'mse': mse, 'r2': r2, 'model_depth': model.get_depth()}
result



While the model shows some predictive capability with a moderate R² value, there is still room for improvement, particularly in reducing the MSE and potentially increasing the explanatory power of the model.

### Exercise 2

In [ ]:


# 1. Correlation Analysis
non_numeric_dataset = dataset.drop(columns = ['Location', 'Soil_type', 'Crop_type'])
correlation_matrix = non_numeric_dataset.corr()
print("Correlation with Standard_yield:\n", correlation_matrix['Standard_yield'].sort_values(ascending=False))

# Plot the correlation matrix as a heatmap
plt.figure(figsize=(12, 10))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f", linewidths=0.5)
plt.title('Correlation Matrix')
plt.show()


Based on the resulting correlation matrix, we would retain the variables with comparatively higher positive or negative absolute correlation coefficients, such as Annual_yield, Min_temperature_C, pH, Pollution_level, and Soil_fertility, and exclude the ones with lower correlation coefficients. It is however important to also use some knowledge we have in how the Standard_yield variable was created – we used Annual_yield as an input, and as such, including it as a predictive variable should be carefully thought about. Let's see what happens if we don't include it for now.  

This will help us retain the most influential features for predicting crop yields while reducing the complexity of the model.

### Exercise 3

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, r2_score

# Define the features to keep based on correlation analysis
features_to_keep = ['Pollution_level','Min_temperature_C','Longitude','pH']

# Prepare the dataset with only the selected features
X = dataset[features_to_keep]
y = dataset['Standard_yield']

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Train the decision tree model
model = DecisionTreeRegressor(max_depth=4, random_state=42).fit(X_train, y_train)

# Make predictions
predictions = model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, predictions)
r2 = r2_score(y_test, predictions)

# Store the results
result = {'mse': mse, 'r2': r2, 'model_depth': model.get_depth()}
result




The model's performance has not improved. The MSE has increased and the R-squared value has decreased. This indicates that using only the selected features has led to a worse predictive performance compared to the previous model. This illustrates the cost of simplified models – we're losing predictive ability whilst gaining ease of interpretation. It's also worth remembering that the R-squared value generally increases as more variables are added – which shouldn't tempt us to do so without good reason!

### Exercise 4

In [ ]:
# Define a range of depths to try
depths_to_try = range(1, 11)

# Initialise empty lists to store results
mse_scores = []
r2_scores = []

# Iterate over depths and fit models
for depth in depths_to_try:
    # Train the model
    model = DecisionTreeRegressor(max_depth=depth, random_state=42).fit(X_train, y_train)
    predictions = model.predict(X_test)
    
    # Evaluate the model
    mse = mean_squared_error(y_test, predictions)
    r2 = r2_score(y_test, predictions)
    
    # Append scores to lists
    mse_scores.append(mse)
    r2_scores.append(r2)

# Print the results
for depth, mse, r2 in zip(depths_to_try, mse_scores, r2_scores):
    print(f"Depth: {depth}, MSE: {mse}, R2: {r2}")


As the depth of the decision tree increases, the MSE generally decreases initially until reaching a minimum at depth 6. Beyond that, the MSE starts to increase again.

The R-squared values also show a similar trend to MSE, initially increasing until depth 6 is reached, and then slightly decreasing afterwards.

Depth 6 seems to provide the best balance between model complexity and performance, as it has the lowest MSE among depths 1 to 5 and a relatively high R² value compared to other depths.

Beyond depth 6, increasing the depth of the decision tree leads to overfitting, as indicated by the increasing MSE and decreasing R-squared values. This means the model starts capturing noise in the data rather than true patterns, resulting in poorer performance on unseen data.

As a final piece of testing (see it as a reward), select only the cassava crops and fit a decision tree to it. Tip: add in the Rainfall variable. The R-squared value is significantly higher than we had for our model. This suggests that we might need to break down the model into decision trees for each crop or use dummy variables when creating it. Sometimes it's useful to approach the same data from different angles, as a fitted model can probably always be improved! 


#  

<div align="center" style=" font-size: 80%; text-align: center; margin: 0 auto">
<img src="https://raw.githubusercontent.com/Explore-AI/Pictures/master/ExploreAI_logos/EAI_Blue_Dark.png"  style="width:200px";/>
</div>